<a href="https://colab.research.google.com/github/Sibaso/Project3_quan.nt173312/blob/main/CL_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyvi
!python -m spacy download en
from pyvi import  ViTokenizer
import spacy
spacy_eng = spacy.load("en")

     |████████████████████████████████| 8.5MB 6.9MB/s 
     |████████████████████████████████| 747kB 57.1MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
from sklearn.model_selection import train_test_split
def data_to_json():
    with open('/content/drive/MyDrive/Project3/data/MT-EV-VLSP2020/basic/data.en', encoding='utf-8') as f:
        english = [line for line in f.read().splitlines()]

    with open('/content/drive/MyDrive/Project3/data/MT-EV-VLSP2020/basic/data.vi', encoding='utf-8') as f:
        vietnamese = [line for line in f.read().splitlines()]
        
    train_src, test_src, train_trg, test_trg = train_test_split(english, vietnamese, train_size=0.8, shuffle=True, random_state=1)
    train_src, valid_src, train_trg, valid_trg = train_test_split(train_src, train_trg, train_size=0.8, shuffle=True, random_state=1)

    df = pd.DataFrame({'eng': train_src, 'viet': train_trg}, columns=['eng', 'viet'])
    df.to_json('/content/drive/My Drive/Project3/data/%s-%s_train.json' % ('ENG', 'VIET'), orient="records", lines=True)

    df = pd.DataFrame({'eng': test_src, 'viet': test_trg}, columns=['eng', 'viet'])
    df.to_json('/content/drive/My Drive/Project3/data/%s-%s_test.json' % ('ENG', 'VIET'), orient="records", lines=True)

    df = pd.DataFrame({'eng': valid_src, 'viet': valid_trg}, columns=['eng', 'viet'])
    df.to_json('/content/drive/My Drive/Project3/data/%s-%s_valid.json' % ('ENG', 'VIET'), orient="records", lines=True)
        
data_to_json()

In [ ]:
from sklearn.model_selection import train_test_split
from pyvi import  ViTokenizer
import spacy
spacy_eng = spacy.load("en")

def preprocess_eng(text):
    text = ' '.join([tok.text for tok in spacy_eng.tokenizer(text)])
    return ' '.join(re.split('\W+', text))

def preprocess_viet(text):
    text = ViTokenizer.tokenize(text)
    return ' '.join(re.split('\W+', text))

def data_to_json(name):
    with open('/content/drive/MyDrive/Project3/data/MT-EV-VLSP2020/indomain-news/%s.en'%name, encoding='utf-8') as f:
        english = [preprocess_eng(line) for line in f.read().splitlines()]

    with open('/content/drive/MyDrive/Project3/data/MT-EV-VLSP2020/indomain-news/%s.vi'%name, encoding='utf-8') as f:
        vietnamese = [preprocess_viet(line) for line in f.read().splitlines()]

    df = pd.DataFrame({'eng': english, 'viet': vietnamese}, columns=['eng', 'viet'])
    df.to_json('/content/drive/My Drive/Project3/data/%s-%s_big_%s.json' % ('eng', 'viet', name), orient="records", lines=True)

        
data_to_json('tst')

In [ ]:
import torchtext

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from collections import defaultdict
from copy import deepcopy
import numpy as np
import pandas as pd
import re
from torchtext.data import Field, BucketIterator, TabularDataset, Dataset, Example
from torchsummary import summary
import math
import copy
from copy import deepcopy
from torch.optim.optimizer import Optimizer
import os
import time
import nltk
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# device='cpu'

cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


EWC

In [ ]:

class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.att_fc = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        # Get number of training examples
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)  # (N, value_len, heads, head_dim)
        keys = self.keys(keys)  # (N, key_len, heads, head_dim)
        queries = self.queries(query)  # (N, query_len, heads, heads_dim)

        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just how I like doing matrix multiplication & bmm

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # energy: (N, heads, query_len, key_len)

        # Mask padded indices so their weights become 0
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # attention shape: (N, heads, query_len, key_len)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out after matrix multiply: (N, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.

        out = self.att_fc(out)
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)
        return out


class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.trb_fc1 = nn.Linear(embed_size, forward_expansion * embed_size)
        self.relu = nn.ReLU()
        self.trb_fc2 = nn.Linear(forward_expansion * embed_size, embed_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.trb_fc1(x)
        forward = self.relu(forward)
        forward = self.trb_fc2(forward)
        out = self.dropout(self.norm2(forward + x))
        return out


class Encoder(nn.Module):
    def __init__(
        self,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        max_length,
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.word_embedding = nn.ModuleDict()
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def add_task(self, src_vocab_size, src_token):
        if src_token in self.word_embedding:
          return
        self.word_embedding[src_token] = nn.Embedding(src_vocab_size, self.embed_size).to(device)

    def forward(self, x, mask, src_token):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(device)
        out = self.dropout(
            (self.word_embedding[src_token](x) + self.position_embedding(positions))
        )

        # In the Encoder the query, key, value are all the same, it's in the
        # decoder this will change. This might look a bit odd in this case.
        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out


class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out


class Decoder(nn.Module):
    def __init__(
        self,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.embed_size = embed_size
        self.word_embedding = nn.ModuleDict()
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.ModuleDict()
        self.dropout = nn.Dropout(dropout)

    def add_task(self, trg_vocab_size, trg_token):
        if trg_token in self.word_embedding:
          return
        self.word_embedding[trg_token] = nn.Embedding(trg_vocab_size, self.embed_size).to(device)
        self.fc_out[trg_token] = nn.Linear(self.embed_size, trg_vocab_size).to(device)

    def forward(self, x, enc_out, src_mask, trg_mask, trg_token):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(device)
        x = self.dropout((self.word_embedding[trg_token](x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out[trg_token](x)

        return F.log_softmax(out, dim=-1)


class Transformer(nn.Module):
    def __init__(
        self,
        src_pad_idx,
        trg_pad_idx,
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        max_length=100,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.softmax = nn.Softmax()

    def add_task(self, src_vocab_size, trg_vocab_size, src_token, trg_token):
        self.encoder.add_task(src_vocab_size, src_token)
        self.decoder.add_task(trg_vocab_size, trg_token)        

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(device)

    def forward(self, src, trg, src_token, trg_token):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask, src_token)
        out = self.decoder(trg, enc_src, src_mask, trg_mask, trg_token)
        return out


In [ ]:

class EWC(object):
    """ Class implementing the Elastic Weight Consolidation approach described in http://arxiv.org/abs/1612.00796 """

    def __init__(self,model, lamb=1000, nepochs=100,sbatch=32,lr=0.001,lr_min=1e-6,lr_factor=3,lr_patience=5):
        self.model=model
        self.model_old=deepcopy(self.get_param(self.model))
        self.fisher=None
        self.tasks = []

        self.nepochs = nepochs
        self.sbatch = sbatch
        self.lr = lr
        self.lr_min = lr_min
        self.lr_factor = lr_factor
        self.lr_patience = lr_patience
        self.lamb = lamb


    def add_task(self, src_vocab_size, trg_vocab_size, src_token, trg_token):
        if {'src': src_token, 'trg': trg_token} in self.tasks:
            return
        self.tasks.append({'src': src_token, 'trg': trg_token})
        self.model.add_task(src_vocab_size, trg_vocab_size, src_token, trg_token)
        self.model = self.model.to(device)

    def load_task(self, t):
        path = '/content/drive/My Drive/Project3/trained_model/ewc_big|{}'.format(
                        '|'.join(['%s-%s'%(task['src'], task['trg']) for task in self.tasks[:t]]))
        self.model.load_state_dict(torch.load('%s.pt'%path))
        self.model.to(device)

    def get_param(self, model):
        sd = []
        for name, param in model.named_parameters():
            if 'word_embedding' in name or 'fc_out' in name or 'position_embedding' in name or 'norm' in name or 'norm1' in name or 'norm2' in name:
                continue 
            sd.append((name, param))
        return sd
   
    def train(self, train_data, valid_data, src_vocab_size, trg_vocab_size, src_token, trg_token):
        best_loss = np.inf
        lr = self.lr
        patience = self.lr_patience
        train_iterator = BucketIterator(train_data, batch_size=self.sbatch, device=device, shuffle=True)
        # valid_iterator = BucketIterator(valid_data, batch_size=self.sbatch, device=device, shuffle=False)
        self.add_task(src_vocab_size, trg_vocab_size, src_token, trg_token)
        self.ce=torch.nn.CrossEntropyLoss(ignore_index=self.model.src_pad_idx)
        self.optimizer=torch.optim.Adam(self.model.parameters(), lr=lr)
        path = '/content/drive/My Drive/Project3/trained_model/ewc_big|{}'.format(
                        '|'.join(['%s-%s'%(task['src'], task['trg']) for task in self.tasks]))
        # Loop epochs
        for e in range(self.nepochs):
            # try:
                # Train
                clock0=time.time()
                train_loss = self.train_epoch(train_iterator,src_token,trg_token)
                clock1=time.time()
                # valid_loss = self.eval(valid_iterator,src_token,trg_token)
                clock2=time.time()
                print('| Epoch {:3d}, time={:5.1f}ms/{:5.1f}ms | Train: loss={:.3f} |'.format(
                    e+1,1000*(clock1-clock0),
                    1000*(clock2-clock1),train_loss),end='')
                # # Valid
                # print(' Valid: loss={:.3f} |'.format(valid_loss),end='')
                
                # # Adapt lr
                # if valid_loss < best_loss:
                #     best_loss = valid_loss
                #     torch.save(self.model.state_dict(),'%s.pt' % path)
                #     patience = self.lr_patience
                #     print(' *', end='')
                
                # else:
                #     patience -= 1
                #     if patience <= 0:
                #         lr /= self.lr_factor
                #         print(' lr={:.1e}'.format(lr), end='')
                #         if lr < self.lr_min:
                #             print()
                            
                #         patience = self.lr_patience
                #         self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
                print()
            # except KeyboardInterrupt:
            #     break

        # Restore best
        torch.save(self.model.state_dict(),'%s.pt' % path)
        # self.model.load_state_dict(torch.load('%s.pt' % path))
        # Update old
        self.model_old = deepcopy(self.get_param(self.model))
        
        # Fisher ops
        if len(self.tasks) > 1:
            fisher_old={}
            for n,_ in self.get_param(self.model):
                fisher_old[n]=self.fisher[n].clone()
        self.fisher=self.fisher_matrix_diag(train_iterator, src_token, trg_token)
        if len(self.tasks) > 1:
            t = len(self.tasks)-1
            # Watch out! We do not want to keep t models (or fisher diagonals) in memory, therefore we have to merge fisher diagonals
            for n,_ in self.get_param(self.model):
                self.fisher[n]=(self.fisher[n]+fisher_old[n]*t)/(t+1)       # Checked: it is better than the other option
                #self.fisher[n]=0.5*(self.fisher[n]+fisher_old[n])

        return

    def train_epoch(self,iterator,src_token,trg_token):
        self.model.train()
        total_loss=0
        # Loop batches
        for batch in iterator:
            src = batch.src.permute(1,0).to(device)
            trg = batch.trg.permute(1,0).to(device)
            output = self.model(src, trg[:, :-1], src_token, trg_token)
            output = output.reshape(-1, output.shape[2])
            trg = trg[:,1:].reshape(-1)   
            loss=self.criterion(output,trg)
            total_loss += loss.item()
            # Backward
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        return total_loss/len(iterator)

    def eval(self,iterator,src_token,trg_token):
        total_loss=0
        self.model.eval()
        # Loop batches
        for batch in iterator:
            src = batch.src.permute(1,0).to(device)
            trg = batch.trg.permute(1,0).to(device)
            with torch.no_grad():
                output = self.model(src, trg[:, :-1], src_token, trg_token)
                output = output.reshape(-1, output.shape[2])
            trg = trg[:,1:].reshape(-1)   
            loss=self.criterion(output,trg)
            total_loss += loss.item()
            
        return total_loss/len(iterator)

    def criterion(self,output,targets):
        # Regularization for all previous tasks
        loss_reg=0
        if len(self.tasks) > 1:
            for (name,param),(_,param_old) in zip(self.get_param(self.model),self.model_old):   
                # print(self.fisher[name].shape, param.shape, param_old.shape)
                loss_reg+=torch.sum(self.fisher[name]*(param_old-param).pow(2))/2
        return self.ce(output,targets)+self.lamb*loss_reg

    def fisher_matrix_diag(self, iterator, src_token, trg_token):
        # Init
        fisher={}
        for n,p in self.get_param(self.model):
            fisher[n]=0*p.data
        # Compute
        self.model.train()
        
        for batch in iterator:
          
            # Forward and backward
            model.zero_grad()
            src = batch.src.permute(1,0).to(device)
            trg = batch.trg.permute(1,0).to(device)
            output = self.model(src, trg[:, :-1], src_token, trg_token)
            output = output.reshape(-1, output.shape[2])
            trg = trg[:,1:].reshape(-1)
            loss=self.criterion(output,trg)
            loss.backward()
            # Get gradients
            for n,p in self.get_param(self.model):
                if p.grad is not None:
                    fisher[n]+=src.shape[0]*p.grad.data.pow(2)
        # Mean
        with torch.no_grad():
            for n,_ in self.get_param(self.model):
                fisher[n]=fisher[n]/len(iterator)
        return fisher

In [ ]:
import random

eng = Field(sequential=True, 
            use_vocab=True,
            init_token='<sos>',
            eos_token = '<eos>',
            lower = True)

viet = Field(sequential=True, 
            use_vocab=True,
            init_token='<sos>',
            eos_token = '<eos>',
            lower = True)

fields = {'eng': ('src', eng), 'viet': ('trg', viet)}
train_eng_viet = TabularDataset('/content/drive/My Drive/Project3/data/%s-%s_big_%s.json'%('eng', 'viet', 'train'), format='json', fields=fields)
test_eng_viet = TabularDataset('/content/drive/My Drive/Project3/data/%s-%s_big_%s.json'%('eng', 'viet', 'tst'), format='json', fields=fields)
# valid_data = TabularDataset('/content/drive/My Drive/Project3/data/%s-%s_valid.json'%('eng', 'viet'), format='json', fields=fields)


eng.build_vocab(train_eng_viet, min_freq=0)

viet.build_vocab(train_eng_viet, min_freq=0)

eng_vocab_size = len(eng.vocab)
viet_vocab_size = len(viet.vocab)
print(eng_vocab_size)
print(viet_vocab_size)

23926
20712


In [ ]:
max_length = max([(len(e.src), len(e.trg)) for e in train_eng_viet.examples])
print(max_length)

(241, 236)


In [ ]:
model = Transformer(
    src_pad_idx=eng.vocab.stoi['<pad>'],
    trg_pad_idx=viet.vocab.stoi['<pad>'],
    embed_size=512,
    num_layers=1,
    forward_expansion=3,
    heads=8,
    dropout=0.1,
    max_length=250,
).to(device)

ewc = EWC(
    model=model,
    lamb=4000,
    nepochs=50,
    sbatch=32,
    lr=0.001,
    lr_min=1e-6,
    lr_factor=3,
    lr_patience=5
)

In [ ]:
ewc.train(train_eng_viet, None, eng_vocab_size, viet_vocab_size, 'eng', 'viet')

| Epoch   1, time=2368.3ms/  0.0ms | Train: loss=0.564 |
| Epoch   2, time=2367.4ms/  0.0ms | Train: loss=0.539 |
| Epoch   3, time=2374.9ms/  0.0ms | Train: loss=0.530 |
| Epoch   4, time=2375.9ms/  0.0ms | Train: loss=0.518 |
| Epoch   5, time=2377.0ms/  0.0ms | Train: loss=0.510 |
| Epoch   6, time=2380.2ms/  0.0ms | Train: loss=0.501 |
| Epoch   7, time=2376.3ms/  0.0ms | Train: loss=0.494 |
| Epoch   8, time=2391.1ms/  0.0ms | Train: loss=0.481 |
| Epoch   9, time=2380.0ms/  0.0ms | Train: loss=0.474 |
| Epoch  10, time=2364.2ms/  0.0ms | Train: loss=0.469 |
| Epoch  11, time=2371.7ms/  0.0ms | Train: loss=0.458 |
| Epoch  12, time=2376.1ms/  0.0ms | Train: loss=0.454 |
| Epoch  13, time=2373.0ms/  0.0ms | Train: loss=0.447 |
| Epoch  14, time=2373.6ms/  0.0ms | Train: loss=0.436 |
| Epoch  15, time=2368.3ms/  0.0ms | Train: loss=0.433 |
| Epoch  16, time=2368.3ms/  0.0ms | Train: loss=0.424 |
| Epoch  17, time=2369.7ms/  0.0ms | Train: loss=0.415 |
| Epoch  18, time=2373.8ms/  0.

In [ ]:
import random

def text_prepocess(text):
    return re.split('\W+', ' '.join(text))

eng = Field(sequential=True, 
            use_vocab=True,
            preprocessing = text_prepocess,
            init_token='<sos>',
            eos_token = '<eos>',
            lower = True)

viet = Field(sequential=True, 
            use_vocab=True,
            preprocessing = text_prepocess,
            init_token='<sos>',
            eos_token = '<eos>',
            lower = True)

fields = {'eng': ('trg', eng), 'viet': ('src', viet)}
train_viet_eng = TabularDataset('/content/drive/My Drive/Project3/data/%s-%s_big_%s.json'%('eng', 'viet', 'train'), format='json', fields=fields)
test_viet_eng = TabularDataset('/content/drive/My Drive/Project3/data/%s-%s_big_%s.json'%('eng', 'viet', 'tst'), format='json', fields=fields)
# valid_data = TabularDataset('/content/drive/My Drive/Project3/data/%s-%s_valid.json'%('eng', 'viet'), format='json', fields=fields)


eng.build_vocab(train_viet_eng, min_freq=0)

viet.build_vocab(train_viet_eng, min_freq=0)

eng_vocab_size = len(eng.vocab)
viet_vocab_size = len(viet.vocab)
print(eng_vocab_size)
print(viet_vocab_size)
print(len(train_viet_eng.examples))

23926
20712
20000


In [ ]:
ewc.train(train_viet_eng, None, viet_vocab_size, eng_vocab_size, 'viet', 'eng')

| Epoch   1, time=2953.2ms/  0.0ms | Train: loss=6.448 |
| Epoch   2, time=2964.2ms/  0.0ms | Train: loss=4.915 |
| Epoch   3, time=2960.2ms/  0.0ms | Train: loss=4.041 |
| Epoch   4, time=2987.6ms/  0.0ms | Train: loss=3.480 |
| Epoch   5, time=2987.8ms/  0.0ms | Train: loss=3.181 |
| Epoch   6, time=2990.3ms/  0.0ms | Train: loss=2.970 |
| Epoch   7, time=2979.9ms/  0.0ms | Train: loss=2.800 |
| Epoch   8, time=2994.0ms/  0.0ms | Train: loss=2.661 |
| Epoch   9, time=2990.1ms/  0.0ms | Train: loss=2.543 |
| Epoch  10, time=2974.0ms/  0.0ms | Train: loss=2.436 |
| Epoch  11, time=2976.0ms/  0.0ms | Train: loss=2.344 |
| Epoch  12, time=2984.4ms/  0.0ms | Train: loss=2.253 |
| Epoch  13, time=2977.3ms/  0.0ms | Train: loss=2.181 |
| Epoch  14, time=2978.2ms/  0.0ms | Train: loss=2.105 |
| Epoch  15, time=2978.6ms/  0.0ms | Train: loss=2.043 |
| Epoch  16, time=2981.9ms/  0.0ms | Train: loss=1.975 |
| Epoch  17, time=2976.5ms/  0.0ms | Train: loss=1.917 |
| Epoch  18, time=2983.7ms/  0.

In [ ]:
def eval_sen(model, src_sen, trg_sen, src, trg, src_lang, trg_lang, max_length=250):

    model.eval()
    src_tensor = src_sen.lower().split()
    src_tensor.insert(0, src.init_token)
    src_tensor.append(src.eos_token)
    src_tensor = [src.vocab.stoi[i] for i in src_tensor]
    src_tensor = torch.LongTensor(src_tensor).unsqueeze(0).to(device)
    outputs = [trg.vocab.stoi["<sos>"]]
    for i in range(max_length):
        translate_tensor = torch.LongTensor(outputs).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(src_tensor, translate_tensor, src_lang, trg_lang)
        
        best_guess = output.argmax(2)[:,-1].item()
        outputs.append(best_guess)

        if best_guess == trg.vocab.stoi["<eos>"]:
            break

    outputs = outputs[1:-1]
    translated_sentence = [trg.vocab.itos[idx] for idx in outputs]
    bleu_score = nltk.translate.bleu_score.sentence_bleu([trg_sen.split(' ')], translated_sentence, weights=(0.5, 0.5))
    # print(translated_sentence)
    # print(trg_sen.split(' '))
    return ' '.join(translated_sentence), bleu_score

def eval(model, data, src_lang, trg_lang, src_token, trg_token):
    total_bleu = 0
    for example in data.examples:
        src_sen = ' '.join(example.src)
        trg_sen = ' '.join(example.trg)
        translated_sentence, bleu_score = eval_sen(model, src_sen, trg_sen, src_lang, trg_lang, src_token, trg_token)
        total_bleu += bleu_score
    return total_bleu / len(data.examples)

In [ ]:
# before learn task 2
print('task 1: eng-viet, bleu score:', eval(model, test_eng_viet, eng, viet, 'eng', 'viet'))

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


task 1: eng-viet, bleu score: 0.31722327460644056


In [ ]:
# after learn task 2
print('task 2: viet-eng, bleu score:', eval(model, test_viet_eng, viet, eng, 'viet', 'eng'))
print('task 1: eng-viet, bleu score:', eval(model, test_eng_viet, eng, viet, 'eng', 'viet'))

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


task 2: viet-eng, bleu score: 0.29126687596523426
task 1: eng-viet, bleu score: 0.28713554416652876


In [ ]:
print('task 1: eng-viet, bleu score:', eval(model, train_eng_viet, eng, viet, 'eng', 'viet'))

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


task 1: eng-viet, bleu score: 0.9543622793442226


In [ ]:
print('task 2: viet-eng, bleu score:', eval(model, train_viet_eng, viet, eng, 'viet', 'eng'))
print('task 1: eng-viet, bleu score:', eval(model, train_eng_viet, eng, viet, 'eng', 'viet'))

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
ewc.add_task(eng_vocab_size, viet_vocab_size, 'eng', 'viet')# task 1
ewc.add_task(viet_vocab_size, eng_vocab_size, 'viet', 'eng')# task 2
ewc.load_task(2)

In [ ]:
# np.random.seed(10)

for example in np.random.choice(test_eng_viet.examples, 10, replace=False):
    src_sen = ' '.join(example.src)
    trg_sen = ' '.join(example.trg)
    translated_sentence, bleu_score = eval_sen(model, src_sen, trg_sen, eng, viet, 'eng', 'viet')
    print()
    print('> source sentence:', src_sen)
    print('< translated sentence:', translated_sentence)
    print('= correct sentence:', trg_sen)
    print('bleu score:',bleu_score)



> source sentence: in addition if you give it enough information to realize that you are a shoe enthusiast the hottest
< translated sentence: ngoài_ra nó nếu bạn hiểu rất đủ lực_lượng để nhận ra cách một cuộc tấn_công sản_phẩm
= correct sentence: ngoài_ra nếu cho nó đủ dữ_kiện để nhận ra rằng bạn là một đầu giày những thông_tin nóng_sốt nhất về thế_giới giày_dép sẽ được hiển_thị đầy_đủ có_vẻ như đây là cung_cách phục_vụ thông_minh dù khách_hàng có tìm đến sản_phẩm của adidas vì chức_năng hay phong_cách
bleu score: 0.04712492027964564

> source sentence: the bun rieu cua is a vermicelli soup with a tomato based broth made by slowly simmering pork or chicken bones topped with fried tofu prawns crab meat bean sprouts and fresh vietnamese herbs like perilla and cilantro
< translated sentence: món ăn này có tên bún riêu cua là một bát nước_dùng cốp cà được chọn bằng cách hành tỏi lá chúc sả trứng và bánh_quy với giá dầu này thông_qua quy_trình này nhằm phục_vụ món ăn này và rau mùi việt_na

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)



> source sentence: it s still hard to be hopeful when you re not seeing anyone doing well at the moment she said
< translated sentence: lyons nói vẫn là mình hy_vọng rằng bạn vẫn chưa biết rằng mình đấy khá rồi cô yêu mình cô đấy
= correct sentence: còn rất khó để hy_vọng khi bạn không nhìn thấy bất_cứ ai đang ổn_định tại thời_điểm hiện_tại cô cho biết
bleu score: 0.4364357804719847

> source sentence: specifically those who develop lung disease within 14 days of returning from wuhan are under surveillance
< translated sentence: cụ_thể cũng đang nỗ_lực những trường_hợp nhiễm_bệnh trong 14 ngày qua tất_cả đều trở về từ vũ_hán
= correct sentence: cụ_thể những người phát bệnh phổi trong vòng 14 ngày kể từ khi trở về từ vũ_hán nằm trong diện theo_dõi
bleu score: 0.2875278132838648

> source sentence: two other babies delivered in da nang and quang nam province in central vietnam weighed 6 5 kilos at birth
< translated sentence: hai trẻ_em khác ở tỉnh quảng_nam và đàn_ông đã hạ_sinh một 5 

In [ ]:

for example in np.random.choice(test_viet_eng.examples, 10, replace=False):
    src_sen = ' '.join(example.src)
    trg_sen = ' '.join(example.trg)
    translated_sentence, bleu_score = eval_sen(model, src_sen, trg_sen, viet, eng, 'viet', 'eng')
    print()
    print('> source sentence:', src_sen)
    print('< translated sentence:', translated_sentence)
    print('= correct sentence:', trg_sen)
    print('bleu score:',bleu_score)


> source sentence: các nhà xuất_khẩu chất_thải nhựa hàng_đầu được phân_tích trong báo_cáo bao_gồm hoa_kỳ anh đức và nhật_bản
< translated sentence: leading plastic waste exporters are the top of the u s report including japan and germany japan
= correct sentence: the top exporters of plastic waste analyzed for the report included the united states britain germany and japan
bleu score: 0.2681514527528423

> source sentence: nó đã được báo_cáo trên the_guardian rằng các siêu_thị của anh và các nhà cung_cấp của họ đã phát_triển một kế_hoạch để đảm_bảo cung_cấp nhất_quán một loạt các hàng_hóa cơ_bản nếu người tiêu_dùng mua hoảng_loạn
< translated sentence: it has been reported that the uk s population and buy a large number of customers developing their plans to ensure a large number of goods companies meet the consumption stimulation programs have buy supply chains buy a consumers
= correct sentence: in early march the guardian reported that british supermarkets and their suppliers had d

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)



> source sentence: vào thứ_bảy trung_tâm kiểm_soát dịch_bệnh ở tỉnh bình_thuận đã cách_ly ba người đàn_ông và một người phụ_nữ trong một cơ_sở tập_trung tại một trường trung_học địa_phương
< translated sentence: in saturday the center for disease control and prevention binh thuan province a man and a woman in a one of a high school year old woman in a local islamic school
= correct sentence: on saturday the center for disease control in binh thuan province quarantined three men and a woman in a centralized facility at a local high school
bleu score: 0.4989909172358461

> source sentence: trong bài phát_biểu của mình ellison cho rằng cuộc biểu_tình là kịch_tính và gần thiết nói rằng floyd đáng_lẽ nên ở đây và giờ thì anh ấy không_thể
< translated sentence: in his statement said that it was that the protest the pandemic and was found that he should be adopted in and his heart and his position
= correct sentence: during his announcement ellison called the protests dramatic and necessary 

In [ ]:
for example in np.random.choice(train_eng_fra.examples, 10, replace=False):
    src_sen = ' '.join(example.src)
    trg_sen = ' '.join(example.trg)
    translated_sentence, bleu_score = eval_sen(model, src_sen, trg_sen, eng1, fra, 'eng1', 'fra')
    print()
    print('> source sentence:', src_sen)
    print('< translated sentence:', translated_sentence)
    print('= correct sentence:', trg_sen)
    print('bleu score:',bleu_score)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)



> source sentence: here catch 
< translated sentence: ici 
= correct sentence: ici attrape 
bleu score: 0.6065306597126334

> source sentence: don t do that 
< translated sentence: ne le pas ça 
= correct sentence: ne fais pas ça 
bleu score: 0.6324555320336759

> source sentence: he dislikes me 
< translated sentence: il ne me ne me pas pas 
= correct sentence: il ne m aime pas 
bleu score: 0.3779644730092272

> source sentence: you re foolish 
< translated sentence: tu es triste 
= correct sentence: tu es imprudente 
bleu score: 0.49999999999999994

> source sentence: tom s scared 
< translated sentence: tom est en train de grandiront 
= correct sentence: tom est apeuré 
bleu score: 0.26726124191242434

> source sentence: i like talking 
< translated sentence: j aime les maison 
= correct sentence: j aime causer 
bleu score: 0.3872983346207417

> source sentence: are we leaving 
< translated sentence: sommes nous 
= correct sentence: y allons nous 
bleu score: 0.41368954504257255

>

In [ ]:
import random

eng = Field(sequential=True, 
            use_vocab=True,
            init_token='<sos>',
            eos_token = '<eos>',
            lower = True)

viet = Field(sequential=True, 
            use_vocab=True,
            init_token='<sos>',
            eos_token = '<eos>',
            lower = True)

fields = {'eng': ('src', eng), 'viet': ('trg', viet)}
train_eng_viet = TabularDataset('/content/drive/My Drive/Project3/data/%s-%s_train.json'%('ENG', 'VIET'), format='json', fields=fields)
test_eng_viet = TabularDataset('/content/drive/My Drive/Project3/data/%s-%s_test.json'%('ENG', 'VIET'), format='json', fields=fields)
# valid_data = TabularDataset('/content/drive/My Drive/Project3/data/%s-%s_valid.json'%('eng', 'viet'), format='json', fields=fields)


eng.build_vocab(train_eng_viet, min_freq=0)

viet.build_vocab(train_eng_viet, min_freq=0)

eng_vocab_size = len(eng.vocab)
viet_vocab_size = len(viet.vocab)
print(eng_vocab_size)
print(viet_vocab_size)

3672
2354
